# loading of clean data into the database

### IMPORTING NECESSARY LIBRARIES 

This section of the code performs the import of the libraries needed to load, manipulate and store data in a relational database. In addition, the working environment is adjusted to allow the import of modules from higher levels of the directory.
The function sys.path.append(os.path.abspath(os.path.join(os.getcwd(), “..”))) adds the parent directory to the sys.path, facilitating the import of modules from higher paths within the project. connect_db (from src.database.db_connection): Provides functions to establish the connection with the database database.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import pandas as pd
from src.database.db_conection import connect_db


In [2]:
engine = connect_db()

Establishes a connection to the database using SQLAlchemy, an ORM (Object Relational Mapper) widely used in data analysis environments. 

This function, located in src.database.db_connection, is responsible for establishing the connection to the database engine. a engine variable stores the connection object returned by connect_db().
This object will allow the execution of SQL queries and interaction with the database from Python.

### loading data into a dataframe

load the rawCandidates table into a dataframe for manipulation and cleaning, as this table has dirty data and some changes are needed for cleaning.

In [3]:
df = pd.read_sql_query('SELECT * FROM "rawCandidates"', engine)
print(df.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

A connection to the database is established and the data stored in the rawCandidates table is extracted.  A dataframe is created to start manipulating the data. In addition to view the first rows of the datset.

### Duplicate data

Before loading data into a destination (database). If duplicates are found in the Object columns, i.e. if a candidate applied more than once, additional measures could be taken such as:

Elimination of duplicates.
Consolidation of similar records.

In [ ]:
duplicates = df[df.duplicated(subset=['First Name', 'Last Name', 'Email'], keep=False)]

if not duplicates.empty:
    print("Duplicate candidates found:")
    print(duplicates)
else:
    print("No duplicate candidates.")


No hay candidatos duplicados.


As we already knew from the same EDA case, there is no duplicate data. 

### Change of data type from Applicate Date column to date type

In [ ]:
df["Application Date"] = pd.to_datetime(df["Application Date"])

### Null data

Analyzes a DataFrame df to detect the presence of null values in each column. This is a fundamental practice in the data cleansing phase of a process, since null values can affect analysis, calculations and data integrity.

In [5]:
null_counts = df.isnull().sum()

if not null_counts.empty:
    print("Valores nulos encontrados:")
    print(null_counts)
else:
    print("No hay valores nulos.")

Valores nulos encontrados:
First Name                   0
Last Name                    0
Email                        0
Application Date             0
Country                      0
YOE                          0
Seniority                    0
Technology                   0
Code Challenge Score         0
Technical Interview Score    0
dtype: int64


There is no null data in any column when the result is displayed. There is no need to delete or do an additional statistic operation to replace. 

### Changing column names

The columns of the DataFrame df were renamed using the Camel Case (camelCase) method. This change is made to improve the understanding and consistency of the data within the process. The normalization of column names facilitates integration with databases, and in the future if the project needs to use APIs or other systems that require a uniform naming structure.

In [6]:
df = df.rename(columns={"First Name": "firstName", 
                        "Last Name": "lastName", 
                        "Email": "email", 
                        "Country": "country",
                        "Application Date": "applicationDate",
                        "YOE": "yearsOfExperience",
                        "Seniority": "seniority",
                        "Technology": "technology",
                        "Code Challenge Score": "codeChallengeScore",
                        "Technical Interview Score": "technicalInterviewScore"})

print(df.head())


    firstName    lastName                      email applicationDate  country  \
0  Bernadette   Langworth        leonard91@yahoo.com      2021-02-26   Norway   
1      Camryn    Reynolds        zelda56@hotmail.com      2021-09-09   Panama   
2       Larue      Spinka   okey_schultz41@gmail.com      2020-04-14  Belarus   
3        Arch      Spinka     elvera_kulas@yahoo.com      2020-10-01  Eritrea   
4       Larue  Altenwerth  minnie.gislason@gmail.com      2020-05-20  Myanmar   

   yearsOfExperience  seniority                         technology  \
0                  2     Intern                      Data Engineer   
1                 10     Intern                      Data Engineer   
2                  4  Mid-Level                     Client Success   
3                 25    Trainee                          QA Manual   
4                 13  Mid-Level  Social Media Community Management   

   codeChallengeScore  technicalInterviewScore  
0                   3                      

By reprinting the first 5 rows, it is evident that the changes in the column names were correctly made. 

### Creation of the hiring column

Since the objective of the project is to analyze the hiring of candidates, it is necessary to clearly identify who was hired. For this purpose, the hiredStatus column was created, which indicates whether a candidate was hired or not.

A candidate in the requirements is considered hired only if both his Code Challenge Score and Technical Interview Score are greater than or equal to 7.  The information becomes clearer for integration with the database and the required visualizations.


In [8]:
df["hiredStatus"] = (df["codeChallengeScore"] >= 7) & (df["technicalInterviewScore"] >= 7)
df["hiredStatus"] = df["hiredStatus"].map({True: "Yes", False: "No"})
print(df.head())  

    firstName    lastName                      email applicationDate  country  \
0  Bernadette   Langworth        leonard91@yahoo.com      2021-02-26   Norway   
1      Camryn    Reynolds        zelda56@hotmail.com      2021-09-09   Panama   
2       Larue      Spinka   okey_schultz41@gmail.com      2020-04-14  Belarus   
3        Arch      Spinka     elvera_kulas@yahoo.com      2020-10-01  Eritrea   
4       Larue  Altenwerth  minnie.gislason@gmail.com      2020-05-20  Myanmar   

   yearsOfExperience  seniority                         technology  \
0                  2     Intern                      Data Engineer   
1                 10     Intern                      Data Engineer   
2                  4  Mid-Level                     Client Success   
3                 25    Trainee                          QA Manual   
4                 13  Mid-Level  Social Media Community Management   

   codeChallengeScore  technicalInterviewScore hiredStatus  
0                   3          

### idCandidates creation 

It is important that the data have a unique identification, as this facilitates its handling in databases and ensures that each candidate can be individually referenced.

Assign a unique identifier to each candidate, which avoids duplication and facilitates data traceability.
Improve the structure of the dataset for its subsequent migration to a relational database.

In [10]:
df["idCandidates"] = df.index + 1 
column_order = ['idCandidates'] + [col for col in df.columns if col != 'idCandidates']
df = df[column_order]

print(df.head())  


   idCandidates   firstName    lastName                      email  \
0             1  Bernadette   Langworth        leonard91@yahoo.com   
1             2      Camryn    Reynolds        zelda56@hotmail.com   
2             3       Larue      Spinka   okey_schultz41@gmail.com   
3             4        Arch      Spinka     elvera_kulas@yahoo.com   
4             5       Larue  Altenwerth  minnie.gislason@gmail.com   

  applicationDate  country  yearsOfExperience  seniority  \
0      2021-02-26   Norway                  2     Intern   
1      2021-09-09   Panama                 10     Intern   
2      2020-04-14  Belarus                  4  Mid-Level   
3      2020-10-01  Eritrea                 25    Trainee   
4      2020-05-20  Myanmar                 13  Mid-Level   

                          technology  codeChallengeScore  \
0                      Data Engineer                   3   
1                      Data Engineer                   2   
2                     Client Success  

### Migration of the transformed dataframe to the database

Once the data was transformed and the columns needed for the analysis were created, it was migrated to a database in a new table called transformedCandidates.

Importance of this migration:
It allows storing the data in a structured and optimized environment for queries.
It facilitates the generation of reports and visualizations based on the database, instead of the original CSV file.
Ensures the persistence and accessibility of the information for future queries and analysis.

In [11]:
df.to_sql("transformedCandidates", engine, if_exists="replace", index=False)
print("Datos insertados correctamente")

Datos insertados correctamente



Data was correctly inserted into the table, ensuring that the structure is consistent with the project objectives.

### Verification of the Data Load in the Database

This step consists of retrieving the data from the transformedCandidates table to verify that the migration was performed correctly and that the changes applied during the process are reflected in the database.

This ensures that: The data was inserted without errors, The transformations performed (such as column renaming, creation of hiredStatus and idCandidates) were applied correctly.

In [12]:
df = pd.read_sql_query('SELECT * FROM "transformedCandidates"', engine)
print(df.head())


   idCandidates   firstName    lastName                      email  \
0             1  Bernadette   Langworth        leonard91@yahoo.com   
1             2      Camryn    Reynolds        zelda56@hotmail.com   
2             3       Larue      Spinka   okey_schultz41@gmail.com   
3             4        Arch      Spinka     elvera_kulas@yahoo.com   
4             5       Larue  Altenwerth  minnie.gislason@gmail.com   

  applicationDate  country  yearsOfExperience  seniority  \
0      2021-02-26   Norway                  2     Intern   
1      2021-09-09   Panama                 10     Intern   
2      2020-04-14  Belarus                  4  Mid-Level   
3      2020-10-01  Eritrea                 25    Trainee   
4      2020-05-20  Myanmar                 13  Mid-Level   

                          technology  codeChallengeScore  \
0                      Data Engineer                   3   
1                      Data Engineer                   2   
2                     Client Success  

The structure of the table is consistent with the project objectives.

A record count is performed on the DataFrame to verify the total amount of data after migration and transformation. This ensures that there is no loss of information during the process and that the database contains the expected number of rows. Which correctly counts the expected number of rows. 

In [ ]:
df.count()

idCandidates               50000
firstName                  50000
lastName                   50000
email                      50000
applicationDate            50000
country                    50000
yearsOfExperience          50000
Seniority                  50000
technology                 50000
codeChallengeScore         50000
technicalInterviewScore    50000
HiredStatus                50000
dtype: int64

### Conclusions and Observations

The process carried out in this project allowed to efficiently transform and migrate the data from a CSV file to a relational database, ensuring the integrity and quality of the information. The key points are highlighted below:

Data Normalization:

Columns were renamed using camelCase to improve consistency and facilitate integration with databases and APIs.
Creation of Essential Columns:

The hiredStatus column was generated to easily identify which candidates were admitted according to the established criteria.
A unique identifier idCandidates was added to facilitate the management and traceability of records in the database.
Problem Data Detection:

Reviewed null values and duplicate data to ensure dataset quality prior to migration.
Migration to the Database:

The transformed data was stored in the transformedCandidates table, ensuring that all modifications made were reflected correctly.
A subsequent query was performed to verify the correct insertion of the data.
Final Validation:

The total number of records after the transformation was counted to confirm that there was no loss of information in the process.